#Using Databricks Foundation model APIs and Databricks Vector Search with LlamaIndex

In [0]:
%pip install mlflow==2.15.*
%pip install --upgrade llama_index
%pip install --upgrade llama-index-llms-databricks  
%pip install --upgrade databricks-vectorsearch
%pip install --upgrade llama-index-vector-stores-databricks
%pip install --upgrade llama-index-embeddings-databricks
dbutils.library.restartPython()  # noqa

In [0]:
import mlflow
mlflow.llama_index.autolog()

In [0]:
from llama_index.core import ServiceContext, VectorStoreIndex, set_global_service_context

# from databricks_llamaindex.databricks_vector_search import DatabricksVectorStore
from llama_index.llms.databricks import Databricks
from llama_index.embeddings.databricks import DatabricksEmbedding
from llama_index.vector_stores.databricks  import DatabricksVectorSearch


from databricks.vector_search.client import (
    VectorSearchIndex,
    VectorSearchClient,
)

In [0]:
# TO DO: Add or update your config

config = {}
config['model_name'] = "databricks-meta-llama-3-1-70b-instruct"
config['embed_model_name'] = "databricks-bge-large-en"
config['index_name'] = '' # TO DO - note, index must use self managed embeddings
config['endpoint_name'] = '' # TO DO

In [0]:
context = dbutils.entry_point.getDbutils().notebook().getContext()  # noqa
token = context.apiToken().get()
host = context.apiUrl().get()
base_url = f'{host}/serving-endpoints/'

llm = Databricks(model=config['model_name'],
          api_key=token,
          api_base=base_url)

embed_model = DatabricksEmbedding(model=config['embed_model_name'],
          api_key=token,
          endpoint=base_url)

In [0]:
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
    context_window=4096,
    num_output=256,
)
set_global_service_context(service_context)

In [0]:
client = VectorSearchClient()
index = client.get_index(endpoint_name="one-env-shared-endpoint-1", index_name="mlops_pj.rr_rag_chatbot.test_new_index")

In [0]:
dvs = DatabricksVectorSearch(
    index = index,
    text_column='content',
    columns = [])
# dvs.add(docs_list)
dvs_index = VectorStoreIndex.from_vector_store(dvs,similarity_top_k=10)
query_engine = dvs_index.as_query_engine(response_mode ='simple_summarize',similarity_top_k=10)

In [0]:
# Query and print response - your trace will be generated here
response = query_engine.query("What is the aircraft type under investigation?")